In [3]:
import pandas as pd
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import bulk, streaming_bulk
import uuid
es = Elasticsearch([{'host': 'localhost', 'port': '9200'}],timeout=30, max_retries=10, retry_on_timeout=True)
#from pandasticsearch import Select
import json
import time
from pandas.io.json import json_normalize

In [53]:
results = helpers.scan(es,
    index="new_myindex",
    scroll = '2m',
    size = 1000,
    preserve_order=True,
    query={'_source': ['content']},
    request_timeout=3600
)
print(type(results))
#from pandasticsearch import Select
#pandas_df = Select.from_dict(results).to_pandas()
#df = pd.concat(map(pd.DataFrame.from_dict, results), axis=1)['_source'].T

<class 'generator'>


In [25]:
def process_hits(hits):
    df=pd.DataFrame()
    for item in hits:
        df = df.append(pd.DataFrame(item)) 
    return df

In [5]:
df = pd.DataFrame()

#Initialize the scroll
#page = es.search(
#  index = 'new_myindex',
 # scroll = '2m',
  #search_type = 'scan',
  #size = 10,
  #body = {})
start = time.time()
page = es.search(
    index='alldataindex',
    #doc_type="_doc",
    scroll='30m',
    size=10000,
    _source=["content"],
    body={},
    request_timeout=10000
)
sid = page['_scroll_id']
scroll_size = len(page['hits']['hits'])

df = df.append(json_normalize(page['hits']['hits']))
#print(scroll_size)
# Start scrolling
i=1
while (scroll_size > 0):
    print("scrolling..",i)
    i=i+1
    page = es.scroll(scroll_id = sid, scroll = '30m')
    #df1 = process_hits(page['hits']['hits'])
    #df=df.append(df1)
    df = df.append(json_normalize(page['hits']['hits']))
    #print(df.head(1))
    # Update the scroll ID
    #es.clear_scroll(sid)
    sid = page['_scroll_id']
    # Get the number of results that we returned in the last scroll
    scroll_size = len(page['hits']['hits'])
    #print("scroll size: " + str(scroll_size))
    # Do something with the obtained page
end = time.time() - start
print(end)
print(len(df))


scrolling.. 1
scrolling.. 2
scrolling.. 3
scrolling.. 4
scrolling.. 5
scrolling.. 6
scrolling.. 7
scrolling.. 8
scrolling.. 9
scrolling.. 10
scrolling.. 11
scrolling.. 12
scrolling.. 13
scrolling.. 14
scrolling.. 15
scrolling.. 16
scrolling.. 17
scrolling.. 18
scrolling.. 19
scrolling.. 20
scrolling.. 21
scrolling.. 22
scrolling.. 23
scrolling.. 24
scrolling.. 25
scrolling.. 26
scrolling.. 27
scrolling.. 28
scrolling.. 29
scrolling.. 30
scrolling.. 31
scrolling.. 32
scrolling.. 33
scrolling.. 34
scrolling.. 35
scrolling.. 36
scrolling.. 37
scrolling.. 38
scrolling.. 39
scrolling.. 40
scrolling.. 41
scrolling.. 42
scrolling.. 43
scrolling.. 44
scrolling.. 45
scrolling.. 46
scrolling.. 47
scrolling.. 48
scrolling.. 49
scrolling.. 50
scrolling.. 51
scrolling.. 52
scrolling.. 53
scrolling.. 54
scrolling.. 55
scrolling.. 56
scrolling.. 57
scrolling.. 58
scrolling.. 59
scrolling.. 60
scrolling.. 61
scrolling.. 62
scrolling.. 63
scrolling.. 64
scrolling.. 65
scrolling.. 66
scrolling.. 67
scro

In [6]:
df.head(5)

,_id,_index,_score,_source.content,_type
0,C7kplWsBcyYnpedmfvCc,alldataindex,1.0,TWO THOUSAND... SIX HUNDRED AND SIXTY-SIX DOLL...,alldataindex_type
1,DLkplWsBcyYnpedmfvCc,alldataindex,1.0,Unleashing your entrepreneurial spirit towards...,alldataindex_type
2,DbkplWsBcyYnpedmfvCc,alldataindex,1.0,When you're one of the country's leaders you p...,alldataindex_type
3,DrkplWsBcyYnpedmfvCc,alldataindex,1.0,An Athlone GAA player owes his life to the qui...,alldataindex_type
4,D7kplWsBcyYnpedmfvCc,alldataindex,1.0,Emerson Electric (NYSE:EMR) has opened bullish...,alldataindex_type


In [2]:
i = 1
p = i/10000
print(p.is_integer())

False


list1=[]
for item in results:
    dict1={}
    dict1['content']= item['_source']['content']
    list1.append(dict1)
df1=pd.DataFrame(list1)
print(len(df1))